In [2]:
import requests
import os
import time
from datetime import datetime, timezone
import pandas as pd
import schedule

RAPID_API_KEY = os.getenv("RAPID_API_KEY")

API_HOST = "adsbexchange-com1.p.rapidapi.com"

URL_TEMP = "https://{HOST}/v2/lat/{LAT}/lon/{LONG}/dist/{DIST}/"

headers = {
    "x-rapidapi-key":RAPID_API_KEY,
    "x-rapidapi-host":API_HOST
}

# Define area of interest

LATITUDE = 27.842490
LONGITUDE = -82.503222
DISTANCE = 50

#initialize database to get data

DATA_DIR = "adsb_data"
os.makedirs(DATA_DIR, exist_ok=True) #creates new directory if needed, otherwise does nothing

#Initialize databse for data with differing number of columns, (or that doesn't fit with focus features)
BAD_DATA_DIR = "adsb_data/weird_data"
os.makedirs(BAD_DATA_DIR, exist_ok=True)

#create a function to get and store data

def get_n_store():
    """Construct API call"""
    api_url = URL_TEMP.format(HOST = API_HOST, LAT = LATITUDE, LONG = LONGITUDE, DIST = DISTANCE)

    headers = {
        "x-rapidapi-key":RAPID_API_KEY,
        "x-rapidapi-hooost":API_HOST
    }

    focus_features = ['hex', 'flight', 't', 'category', 'alt_baro', 'alt_geom', 'gs',
       'track', 'baro_rate','geom_rate', 'emergency', 'lat', 'lon',
       'nac_p', 'nac_v','alert','seen', 'nav_altitude_mcp',
       'nav_heading', 'true_heading']

    try:
        response = requests.get(api_url,headers=headers)
        #Parse response
        adsb_data = response.json().get("ac",[])

        if not adsb_data:
            return(f"{datetime.now(timezone.utc)}: No data returned.")
        """Try to get only the features we're interested in, and make sure that 
        there are a correct number of columns."""
        
        try:
            df = pd.DataFrame(adsb_data).loc[:, focus_features]
            bad_data_path = os.path.join(BAD_DATA_DIR, "bad_data.csv")
            
            if (df.columns != focus_features).all():
                #Don't want to waste an api call on data that's missing a column or two
                df_weird = pd.DataFrame(adsb_data)
                df_weird['timestamp'] = datetime.now(timezone.utc)

                if not os.path.isfile(bad_data_path):
                    df_weird.to_csv(bad_data_path, index=False, mode="w",header=True)
                else:
                    df_weird.to_csv(bad_data_path, index=False, mode="a", header=False)

        except Exception as e:
            print(f"Feature selection error: {e}")

        if not df.shape[1] == 20:
            return("df.shape[1] != 20")
        
        df['timestamp'] = datetime.now(timezone.utc)
        df['mph'] = (df['gs'] * 1.151)

        file_path = os.path.join(DATA_DIR, "adsb_data.csv")

        if not os.path.isfile(file_path):
            df.to_csv(file_path, index=False, mode="w",header=True)
        else:
            df.to_csv(file_path, index=False, mode="a", header=False)

        print(f"{datetime.now(timezone.utc)}: Data fetched and stored!")


    except Exception as e:
        #returns the error with the time it occurred
        return(f'{datetime.now(timezone.utc)}: An error occured: ({e})')

In [3]:
file = pd.read_csv("./adsb_data/adsb_data.csv")

In [5]:
print(len(file))


20215


In [8]:
file.columns

Index(['hex', 'flight', 't', 'category', 'alt_baro', 'alt_geom', 'gs', 'track',
       'baro_rate', 'geom_rate', 'emergency', 'lat', 'lon', 'nac_p', 'nac_v',
       'alert', 'seen', 'nav_altitude_mcp', 'nav_heading', 'true_heading',
       'timestamp', 'mph'],
      dtype='object')

In [9]:
file.head()

,hex,flight,t,category,alt_baro,alt_geom,gs,track,baro_rate,geom_rate,...,lon,nac_p,nac_v,alert,seen,nav_altitude_mcp,nav_heading,true_heading,timestamp,mph
0,a98bef,N714LM,C525,A1,16025,16825.0,316.9,155.78,-2496.0,NaN,...,-83.162519,10.0,2.0,0.0,0.0,12992.0,NaN,NaN,2024-10-01 19:41:01.080409+00:00,364.7519
1,a8d600,DAL1196,B752,A4,38000,40225.0,466.0,3.81,64.0,NaN,...,-83.133703,10.0,2.0,0.0,0.2,38016.0,4.92,NaN,2024-10-01 19:41:01.080409+00:00,536.3660
2,a9036a,N680CA,BE20,A1,6025,6250.0,191.4,63.30,0.0,NaN,...,-83.105868,10.0,2.0,0.0,0.1,6016.0,NaN,NaN,2024-10-01 19:41:01.080409+00:00,220.3014
3,a416be,UAL2246,B738,A3,30125,32025.0,440.2,162.14,-2112.0,NaN,...,-83.078506,8.0,1.0,0.0,0.0,27008.0,130.08,NaN,2024-10-01 19:41:01.080409+00:00,506.6702
4,a88b70,N65LW,C25B,A1,7575,7850.0,243.6,156.28,NaN,-1600.0,...,-83.022542,10.0,1.0,0.0,0.1,6016.0,180.00,NaN,2024-10-01 19:41:01.080409+00:00,280.3836
